In [24]:
import pandas as pd

#参数初始化
inputfile = './BLE_RSSI_dataset/iBeacon_RSSI_UnLabeled.csv' #销量及其他属性数据
outputfile = './BLE_RSSI_dataset/iBeacon_RSSI_PreLabeled.csv' #保存结果的文件名
k = 3 #聚类的类别
iteration = 3 #聚类最大循环次数
data = pd.read_csv(inputfile) #读取数据
#data_zs = 1.0*(data - data.mean())/data.std() #数据标准化，std()表示求总体样本方差(除以n-1),numpy中std()是除以n
#print(data_zs)
data = data.drop(columns=['location','date'])

from sklearn.cluster import KMeans
model = KMeans(n_clusters = k, max_iter = iteration) #分为k类
#model = KMeans(n_clusters = k, n_jobs = 4, max_iter = iteration) #分为k类，并发数4
model.fit(data) #开始聚类

#简单打印结果
r1 = pd.Series(model.labels_).value_counts() #统计各个类别的数目
r2 = pd.DataFrame(model.cluster_centers_) #找出聚类中心
r = pd.concat([r2, r1], axis = 1) #横向连接（0是纵向），得到聚类中心对应的类别下的数目
print(r)
r.columns = list(data.columns) + [u'类别数目'] #重命名表头
print(r)

#详细输出原始数据及其类别
r = pd.concat([data, pd.Series(model.labels_, index = data.index)], axis = 1)  #详细输出每个样本对应的类别
r.columns = list(data.columns) + [u'聚类类别'] #重命名表头
#r.to_excel(outputfile) #保存结果

           0           1           2           3           4           5   \
0 -196.987631 -181.876308 -168.901998 -134.963844 -186.702188 -118.604186   
1 -171.853812 -195.613004 -198.596413 -199.957399 -196.481166 -196.975785   
2 -178.094241 -123.324607 -167.445026 -191.305236  -97.413089 -133.178010   

           6           7           8           9           10          11  \
0 -135.089439 -194.586108 -200.000000 -199.540438 -199.795433 -200.000000   
1 -199.765919 -184.315247 -164.332287 -174.055605 -176.470404 -177.591480   
2 -196.164398 -137.185864 -199.052356 -191.030366 -194.534555 -199.169634   

           12    0   
0 -200.000000  1083  
1 -172.667265  2196  
2 -199.945550  1912  
        b3001       b3002       b3003       b3004       b3005       b3006  \
0 -196.987631 -181.876308 -168.901998 -134.963844 -186.702188 -118.604186   
1 -171.853812 -195.613004 -198.596413 -199.957399 -196.481166 -196.975785   
2 -178.094241 -123.324607 -167.445026 -191.305236  -97.413089 -

In [ ]:
#用TSNE进行数据降维并展示聚类结果
from sklearn.manifold import TSNE
tsne = TSNE()
tsne.fit_transform(data) #进行数据降维,并返回结果
tsne = pd.DataFrame(tsne.embedding_, index = data.index) #转换数据格式

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

#不同类别用不同颜色和样式绘图
d = tsne[r[u'聚类类别'] == 0]     #找出聚类类别为0的数据对应的降维结果
plt.plot(d[0], d[1], 'r.')
d = tsne[r[u'聚类类别'] == 1]
plt.plot(d[0], d[1], 'go')
d = tsne[r[u'聚类类别'] == 2]
plt.plot(d[0], d[1], 'b*')
plt.show()